# Connect 4

---

Author: S. Menary [sbmenary@gmail.com]

Date  : 2023-01-18, last edit 2023-01-19

Brief : Play a tournment between two bots to determine which is stronger

---

## Imports

In [1]:
##=====================================##
##  All imports should be placed here  ##
##=====================================##

##  Python core libs
import pickle, sys, time

##  PyPI libs
import numpy as np

##  Local packages
from connect4.utils    import DebugLevel
from connect4.game     import BinaryPlayer, GameBoard, GameResult
from connect4.MCTS     import PolicyStrategy
from connect4.bot      import Bot_NeuralMCTS, Bot_VanillaMCTS
from connect4.parallel import generate_from_processes


In [2]:
##=====================================##
##  Print version for reproducibility  ##
##=====================================##

print(f"Python version is {sys.version}")
print(f"Numpy  version is {np.__version__}")


Python version is 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Numpy  version is 1.23.2


##  MCTS

The `Bot_NeuralMCTS` object is used to apply bot actions using MCTS with a loaded model.

In [3]:
##=========================================##
##  Define methods for playing tournament  ##
##=========================================##


def play_bot_game(model1, model2, duration:int=1, discount=1., debug_lvl:DebugLevel=DebugLevel.MUTE) :
    ##  Create game and bots
    policy_strategy = PolicyStrategy.SAMPLE_POSTERIOR_POLICY
    game_board = GameBoard()
    bot1       = Bot_NeuralMCTS(model1, policy_strategy=policy_strategy) if model1 else Bot_VanillaMCTS(policy_strategy=policy_strategy)
    bot2       = Bot_NeuralMCTS(model2, policy_strategy=policy_strategy) if model2 else Bot_VanillaMCTS(policy_strategy=policy_strategy)
    debug_lvl.message(DebugLevel.LOW, f"Using bot1 {bot1}")
    debug_lvl.message(DebugLevel.LOW, f"Using bot2 {bot2}")
    debug_lvl.message(DebugLevel.LOW, game_board)
    
    ##  Shuffle bots
    ret = {"model1":BinaryPlayer.X, "model2":BinaryPlayer.O}
    if np.random.choice([True, False]) :
        bot1, bot2 = bot2, bot1
        ret = {"model1":BinaryPlayer.O, "model2":BinaryPlayer.X}

    ##  Take moves until end of game
    result = game_board.get_result()
    while not result :
        bot = bot1 if game_board.to_play == BinaryPlayer.X else bot2
        bot.take_move(game_board, duration=duration, discount=discount, debug_lvl=debug_lvl)
        result = game_board.get_result()
        
    ##  Return
    ret["result"] = result
    return ret


def bot_tournament_process(proc_idx, num_games, out_queue, argv) :
    from connect4.neural import load_model
    model1_name, model2_name, duration, discount, base_seed = argv
    np.random.seed(base_seed+proc_idx)
    model1 = load_model(model1_name) if len(model1_name) > 0 else None
    model2 = load_model(model2_name) if len(model2_name) > 0 else None
    for game_idx in range(num_games) :
        _ = play_bot_game(model1, model2, duration, discount)
        out_queue.put(_)


In [4]:
##================================##
##  Specify models and filenames  ##
##================================##

idx_old = 0
idx_new = 4

model_name_old = f"../models/.neural_model_v{idx_old}.h5" if idx_old else ""
model_name_new = f"../models/.neural_model_v{idx_new}.h5" if idx_new else ""
results_fname  = f"../data/.bot_tournament_v{idx_old}_vs_v{idx_new}.pickle"

print(f"Defining old model: {model_name_old}")
print(f"Defining new model: {model_name_new}")
print(f"      Results file: {results_fname}")


Defining old model: 
Defining new model: ../models/.neural_model_v4.h5
      Results file: ../data/.bot_tournament_v0_vs_v4.pickle


In [5]:
##=====================================================================##
##  Load old results, or run parallel subprocesses to obtain new ones  ##
##=====================================================================##

load_from_file = False

num_proc           = 7
num_games_per_proc = 10
base_seed          = 10
mon_freq           = 3
duration           = 5
discount           = .99

if load_from_file :
    print(f"Loading past results from file {results_fname}")
    loaded  = pickle.load(open(results_fname, "rb"))
    results = loaded["results"]
    for key, val in loaded.items() :
        if key == "results" : continue
        print(f"  Found saved config value {key} = {val}")
else :
    print(f"Generating {num_proc*num_games_per_proc} results with base seed {base_seed}")
    print(f"Using duration = {duration:.3f}\nUsing discount = {discount:.3f}")
    results = generate_from_processes(
        func      = bot_tournament_process,
        func_args = [model_name_old, model_name_new, duration, discount, base_seed],
        num_proc  = num_proc, 
        num_results_per_proc = num_games_per_proc, 
        mon_freq  = mon_freq)


Generating 70 results with base seed 10
Using duration = 5.000
Using discount = 0.990
Generated 70 / 70 results [t=1906.76s]
Generation complete [t=1906.76s] [n=70]


In [6]:
##========================##
##  Report on the scores  ##
##========================##

num_results = len(results)
score = [[0, 0, 0, 0], [0, 0, 0, 0]]

for result in results :
    score_idx = 0
    score_idx = 0 if result["model1"] == BinaryPlayer.X else 1
    result_idx = 0
    if result["result"] == GameResult.X    : result_idx = 1
    if result["result"] == GameResult.O    : result_idx = 2
    if result["result"] == GameResult.DRAW : result_idx = 3
    score[score_idx][result_idx] += 1
    
print(f"Results (LHS player goes first): Old Model {score[0][1]} - {score[0][2]} New Model")
print(f"Results (LHS player goes first): New Model {score[1][1]} - {score[1][2]} Old Model")
    

Results (LHS player goes first): Old Model 8 - 26 New Model
Results (LHS player goes first): New Model 33 - 2 Old Model


In [7]:
##====================================##
##  Save results for later reference  ##
##====================================##

print(f"Saving results to file {results_fname}")
to_save = {"results"     : results,
           "num_results" : num_results,
           "base_seed"   : base_seed,
           "duration"    : duration,
           "discount"    : discount}
pickle.dump(to_save, open(results_fname, "wb"))


Saving results to file ../data/.bot_tournament_v0_vs_v4.pickle
